In [1]:
import tempfile
import sys
import subprocess
import shutil
from os.path import join
from elasticsearch import Elasticsearch, helpers
from nir.utils import create_filter_query_function, change_bm25_parameters
from utils import *
import pickle


import json

es = Elasticsearch(["http://193.136.175.98:8125"])

index_name = "bioasq_9b"


In [2]:
queries = load_queries("training9b_wDates.json", maps=[("body","query")])

test_8b = []

for i in range(1,6):
    id_sets = set()
    with open(f"yearly_data/8B{i}_golden.json","r") as f:
        for q in json.load(f)["questions"]:
            id_sets.add(q["id"])
            
    test_8b.append(id_sets)

print(sum([len(x) for x in test_8b]))



500


## Round 1

In [ ]:
queries_8b1 = subset_byId(queries, test_8b[0])

queries_8b1, queries_8b1_gs = separate_queries_goldstandard(queries_8b1, additional_keys=["limit_date"])
print(len(queries_8b1))

In [3]:
import itertools
import random

## SEARCH PARAMETERS

TOP_N = [10,100, 250, 500]
K1 = list(map(lambda x:x/10, range(0,11,1)))
B = list(map(lambda x:x/100, range(0,101,1)))

RANDOM_SEARCH = True

combinations = list(itertools.product(K1,B))

if RANDOM_SEARCH:
    N_Experiments = 500
    selected = random.sample(combinations, N_Experiments)
    print(selected)
else:
    selected = combinations

count = 0
with open("search_bm25_bioasq_9b_batch_01"+".csv", "w") as fOut:
    header = "k1,b,"+",".join(sum([ ["map@"+str(x),"recall@"+str(x)] for x in TOP_N],[]))+"\n"
    fOut.write(header)
    
    for k1,beta in selected:
        
        print(count,"/",len(selected))
        
        results = execute_search(es, queries_8b1, TOP_N[-1], index_name, k1=k1, b=beta)
        results = create_document_run(queries_8b1, results)
        
        metrics_list = map(lambda x:str(x), sum([ list(evaluation(results, queries_8b1_gs, top)) for top in TOP_N],[]))
        line = f"{k1},{beta}," + ",".join(metrics_list)+"\n"
        fOut.write(line)
        fOut.flush()
        count+=1
        
        

[(0.5, 0.51), (0.3, 0.42), (0.7, 0.36), (0.7, 0.75), (0.9, 0.36), (0.3, 0.53), (1.0, 0.8), (0.3, 0.27), (0.7, 0.78), (0.3, 0.2), (0.7, 0.95), (0.0, 0.67), (1.0, 0.07), (0.2, 0.36), (1.0, 0.55), (0.0, 0.42), (0.8, 0.3), (0.0, 0.47), (0.6, 0.32), (0.4, 0.74), (0.7, 0.98), (0.0, 0.25), (0.8, 0.25), (0.9, 0.2), (0.2, 0.75), (0.0, 0.81), (0.2, 0.65), (0.1, 0.44), (0.1, 0.6), (1.0, 0.31), (0.3, 0.87), (0.5, 0.84), (0.5, 0.1), (0.0, 0.31), (0.0, 0.05), (0.8, 0.86), (0.1, 0.32), (0.1, 0.1), (0.7, 0.63), (0.8, 0.09), (1.0, 0.01), (0.2, 0.12), (0.9, 0.41), (0.5, 0.63), (0.8, 0.16), (0.8, 0.82), (0.0, 0.77), (0.7, 0.38), (0.4, 0.69), (0.0, 0.51), (0.4, 0.11), (0.4, 0.4), (0.0, 0.79), (0.9, 0.25), (0.7, 0.08), (0.1, 0.34), (0.5, 0.97), (0.2, 0.31), (0.0, 0.64), (1.0, 0.88), (0.0, 0.91), (0.6, 0.9), (0.0, 0.76), (0.8, 0.42), (0.5, 0.67), (0.2, 0.16), (1.0, 0.48), (0.8, 0.51), (1.0, 0.44), (0.5, 0.2), (1.0, 0.18), (0.9, 0.53), (0.8, 0.61), (0.5, 0.32), (0.2, 0.93), (0.7, 0.45), (0.0, 0.45), (1.0, 0.

37 / 500query: 80
Setting the k1 and b for BM25
38 / 500query: 80
Setting the k1 and b for BM25
39 / 500query: 80
Setting the k1 and b for BM25
40 / 500query: 80
Setting the k1 and b for BM25
41 / 500query: 80
Setting the k1 and b for BM25
42 / 500query: 80
Setting the k1 and b for BM25
43 / 500query: 80
Setting the k1 and b for BM25
44 / 500query: 80
Setting the k1 and b for BM25
45 / 500query: 80
Setting the k1 and b for BM25
46 / 500query: 80
Setting the k1 and b for BM25
47 / 500query: 80
Setting the k1 and b for BM25
48 / 500query: 80
Setting the k1 and b for BM25
49 / 500query: 80
Setting the k1 and b for BM25
50 / 500query: 80
Setting the k1 and b for BM25
51 / 500query: 80
Setting the k1 and b for BM25
52 / 500query: 80
Setting the k1 and b for BM25
53 / 500query: 80
Setting the k1 and b for BM25
54 / 500query: 80
Setting the k1 and b for BM25
55 / 500query: 80
Setting the k1 and b for BM25
56 / 500query: 80
Setting the k1 and b for BM25
57 / 500query: 80
Setting the k1 and b f

KeyboardInterrupt: 

In [4]:
## Visualizing results

import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_01.csv")
df.sort_values('recall@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
91,0.9,0.40,0.344051,0.435528,0.403558,0.706153,0.404921,0.758365,0.405488,0.821070
68,1.0,0.44,0.342657,0.431916,0.403810,0.705439,0.405265,0.761350,0.405802,0.826070
42,0.9,0.41,0.344630,0.438861,0.403075,0.705439,0.404568,0.758365,0.405134,0.821070
109,0.9,0.42,0.346192,0.438861,0.404626,0.705439,0.406110,0.758365,0.406675,0.821070
29,1.0,0.31,0.337836,0.434935,0.398476,0.705403,0.399866,0.755065,0.400438,0.826070
...,...,...,...,...,...,...,...,...,...,...
21,0.0,0.25,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
153,0.0,0.59,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
17,0.0,0.47,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503
15,0.0,0.42,0.284669,0.406183,0.345025,0.622416,0.345792,0.660698,0.346386,0.737503


## Round 2


In [4]:
queries_8b2 = subset_byId(queries, test_8b[1])

queries_8b2, queries_8b2_gs = separate_queries_goldstandard(queries_8b2, additional_keys=["limit_date"])
print(len(queries_8b2))

100


In [ ]:
import itertools
import random

## SEARCH PARAMETERS

TOP_N = [10,100, 250, 500]
K1 = list(map(lambda x:x/10, range(0,11,1)))
B = list(map(lambda x:x/100, range(0,101,1)))

RANDOM_SEARCH = False

combinations = list(itertools.product(K1,B))

if RANDOM_SEARCH:
    N_Experiments = 500
    selected = random.sample(combinations, N_Experiments)
    print(selected)
else:
    selected = combinations

count = 0
with open("search_bm25_bioasq_9b_batch_01"+".csv", "w") as fOut:
    header = "k1,b,"+",".join(sum([ ["map@"+str(x),"recall@"+str(x)] for x in TOP_N],[]))+"\n"
    fOut.write(header)
    
    for k1,beta in selected:
        
        print(count,"/",len(selected))
        
        results = execute_search(es, queries_8b2, TOP_N[-1], index_name, k1=k1, b=beta)
        results = create_document_run(queries_8b2, results)
        
        metrics_list = map(lambda x:str(x), sum([ list(evaluation(results, queries_8b2_gs, top)) for top in TOP_N],[]))
        line = f"{k1},{beta}," + ",".join(metrics_list)+"\n"
        fOut.write(line)
        fOut.flush()
        count+=1
        
        

0 / 1111
Setting the k1 and b for BM25
1 / 1111query: 80
Setting the k1 and b for BM25
2 / 1111query: 80
Setting the k1 and b for BM25
3 / 1111query: 80
Setting the k1 and b for BM25
4 / 1111query: 80
Setting the k1 and b for BM25
5 / 1111query: 80
Setting the k1 and b for BM25
6 / 1111query: 80
Setting the k1 and b for BM25
7 / 1111query: 80
Setting the k1 and b for BM25
8 / 1111query: 80
Setting the k1 and b for BM25
9 / 1111query: 80
Setting the k1 and b for BM25
10 / 1111uery: 80
Setting the k1 and b for BM25
11 / 1111uery: 80
Setting the k1 and b for BM25
12 / 1111uery: 80
Setting the k1 and b for BM25
13 / 1111uery: 80
Setting the k1 and b for BM25
14 / 1111uery: 80
Setting the k1 and b for BM25
15 / 1111uery: 80
Setting the k1 and b for BM25
16 / 1111uery: 80
Setting the k1 and b for BM25
17 / 1111uery: 80
Setting the k1 and b for BM25
18 / 1111uery: 80
Setting the k1 and b for BM25
19 / 1111uery: 80
Setting the k1 and b for BM25
20 / 1111uery: 80
Setting the k1 and b for BM25
2

171 / 1111ery: 80
Setting the k1 and b for BM25
172 / 1111ery: 80
Setting the k1 and b for BM25
173 / 1111ery: 80
Setting the k1 and b for BM25
174 / 1111ery: 80
Setting the k1 and b for BM25
175 / 1111ery: 80
Setting the k1 and b for BM25
176 / 1111ery: 80
Setting the k1 and b for BM25
177 / 1111ery: 80
Setting the k1 and b for BM25
178 / 1111ery: 80
Setting the k1 and b for BM25
179 / 1111ery: 80
Setting the k1 and b for BM25
180 / 1111ery: 80
Setting the k1 and b for BM25
181 / 1111ery: 80
Setting the k1 and b for BM25
182 / 1111ery: 80
Setting the k1 and b for BM25
183 / 1111ery: 80
Setting the k1 and b for BM25
184 / 1111ery: 80
Setting the k1 and b for BM25
185 / 1111ery: 80
Setting the k1 and b for BM25
186 / 1111ery: 80
Setting the k1 and b for BM25
187 / 1111ery: 80
Setting the k1 and b for BM25
188 / 1111ery: 80
Setting the k1 and b for BM25
189 / 1111ery: 80
Setting the k1 and b for BM25
190 / 1111ery: 80
Setting the k1 and b for BM25
191 / 1111ery: 80
Setting the k1 and b f

342 / 1111ery: 80
Setting the k1 and b for BM25
343 / 1111ery: 80
Setting the k1 and b for BM25
344 / 1111ery: 80
Setting the k1 and b for BM25
345 / 1111ery: 80
Setting the k1 and b for BM25
346 / 1111ery: 80
Setting the k1 and b for BM25
347 / 1111ery: 80
Setting the k1 and b for BM25
348 / 1111ery: 80
Setting the k1 and b for BM25
349 / 1111ery: 80
Setting the k1 and b for BM25
350 / 1111ery: 80
Setting the k1 and b for BM25
351 / 1111ery: 80
Setting the k1 and b for BM25
352 / 1111ery: 80
Setting the k1 and b for BM25
353 / 1111ery: 80
Setting the k1 and b for BM25
354 / 1111ery: 80
Setting the k1 and b for BM25
355 / 1111ery: 80
Setting the k1 and b for BM25
356 / 1111ery: 80
Setting the k1 and b for BM25
357 / 1111ery: 80
Setting the k1 and b for BM25
358 / 1111ery: 80
Setting the k1 and b for BM25
359 / 1111ery: 80
Setting the k1 and b for BM25
360 / 1111ery: 80
Setting the k1 and b for BM25
361 / 1111ery: 80
Setting the k1 and b for BM25


In [7]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_01.csv")
df.sort_values('recall@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
370,0.3,0.67,0.283801,0.468481,0.353207,0.792399,0.356148,0.853968,0.356796,0.887221
368,0.3,0.65,0.284170,0.468481,0.353520,0.792399,0.356469,0.853968,0.357120,0.887221
369,0.3,0.66,0.284354,0.468481,0.353790,0.792399,0.356731,0.853968,0.357380,0.887221
377,0.3,0.74,0.282593,0.469084,0.351026,0.791639,0.354136,0.852422,0.354890,0.887221
371,0.3,0.68,0.282072,0.468481,0.351378,0.791630,0.354446,0.853968,0.355093,0.887221
...,...,...,...,...,...,...,...,...,...,...
71,0.0,0.71,0.226392,0.370455,0.289919,0.713360,0.293313,0.806913,0.294015,0.849995
72,0.0,0.72,0.226392,0.370455,0.289919,0.713360,0.293313,0.806913,0.294015,0.849995
73,0.0,0.73,0.226392,0.370455,0.289919,0.713360,0.293313,0.806913,0.294015,0.849995
74,0.0,0.74,0.226392,0.370455,0.289919,0.713360,0.293313,0.806913,0.294015,0.849995


## Round 3


In [3]:
queries_8b3 = subset_byId(queries, test_8b[2])

queries_8b3, queries_8b3_gs = separate_queries_goldstandard(queries_8b3, additional_keys=["limit_date"])
print(len(queries_8b3))

100


In [ ]:
import itertools
import random

## SEARCH PARAMETERS

TOP_N = [10,100, 250, 500]
K1 = list(map(lambda x:x/10, range(0,11,1)))
B = list(map(lambda x:x/100, range(0,101,1)))

RANDOM_SEARCH = False

combinations = list(itertools.product(K1,B))

if RANDOM_SEARCH:
    N_Experiments = 500
    selected = random.sample(combinations, N_Experiments)
    print(selected)
else:
    selected = combinations

count = 0
with open("search_bm25_bioasq_9b_batch_03"+".csv", "w") as fOut:
    header = "k1,b,"+",".join(sum([ ["map@"+str(x),"recall@"+str(x)] for x in TOP_N],[]))+"\n"
    fOut.write(header)
    
    for k1,beta in selected:
        
        print(count,"/",len(selected))
        
        results = execute_search(es, queries_8b3, TOP_N[-1], index_name, k1=k1, b=beta)
        results = create_document_run(queries_8b3, results)
        
        metrics_list = map(lambda x:str(x), sum([ list(evaluation(results, queries_8b3_gs, top)) for top in TOP_N],[]))
        line = f"{k1},{beta}," + ",".join(metrics_list)+"\n"
        fOut.write(line)
        fOut.flush()
        count+=1
        
        

0 / 1111
Setting the k1 and b for BM25
The inquery limit_date will be used
1 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
2 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
3 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
4 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
5 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
6 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
7 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
8 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
9 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
10 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
11 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
1

98 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
99 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
100 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
101 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
102 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
103 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
104 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
105 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
106 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
107 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
108 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
109 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will 

The inquery limit_date will be used
196 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
197 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
198 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
199 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
200 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
201 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
202 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
203 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
204 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
205 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
206 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
207 / 1111ery: 80
Setting the k1 and b f

The inquery limit_date will be used
391 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
392 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
393 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
394 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
395 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
396 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
397 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
398 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
399 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
400 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
401 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
402 / 1111ery: 80
Setting the k1 and b f

The inquery limit_date will be used
586 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
587 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
588 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
589 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
590 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
591 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
592 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
593 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
594 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
595 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
596 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
597 / 1111ery: 80
Setting the k1 and b f

The inquery limit_date will be used
781 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
782 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
783 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
784 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
785 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
786 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
787 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
788 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
789 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
790 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
791 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
792 / 1111ery: 80
Setting the k1 and b f

In [5]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_03.csv")
df.sort_values('recall@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
584,0.5,0.79,0.430934,0.541922,0.495930,0.796195,0.497171,0.829154,0.497711,0.855632
576,0.5,0.71,0.432882,0.539875,0.498656,0.796195,0.499929,0.829154,0.500477,0.855632
577,0.5,0.72,0.433082,0.540375,0.498542,0.796195,0.499808,0.829154,0.500355,0.855632
578,0.5,0.73,0.433419,0.540375,0.499018,0.796195,0.500280,0.829154,0.500827,0.855632
579,0.5,0.74,0.432842,0.540375,0.498495,0.796195,0.499759,0.829154,0.500305,0.855632
...,...,...,...,...,...,...,...,...,...,...
71,0.0,0.71,0.349747,0.460844,0.404347,0.722917,0.406382,0.788450,0.407064,0.839168
72,0.0,0.72,0.349747,0.460844,0.404347,0.722917,0.406382,0.788450,0.407064,0.839168
73,0.0,0.73,0.349747,0.460844,0.404347,0.722917,0.406382,0.788450,0.407064,0.839168
74,0.0,0.74,0.349747,0.460844,0.404347,0.722917,0.406382,0.788450,0.407064,0.839168


In [6]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_03.csv")
df.sort_values('map@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
918,0.9,0.09,0.470780,0.544755,0.533291,0.789167,0.534779,0.83596,0.535150,0.862453
909,0.9,0.00,0.470070,0.542557,0.532110,0.787119,0.533544,0.83221,0.533946,0.857703
917,0.9,0.08,0.468555,0.543986,0.531223,0.786667,0.532801,0.83596,0.533151,0.860453
916,0.9,0.07,0.468950,0.545236,0.530685,0.786667,0.532259,0.83596,0.532607,0.860453
914,0.9,0.05,0.468881,0.543986,0.530658,0.786667,0.532183,0.83346,0.532565,0.860453
...,...,...,...,...,...,...,...,...,...,...
71,0.0,0.71,0.349747,0.460844,0.404347,0.722917,0.406382,0.78845,0.407064,0.839168
72,0.0,0.72,0.349747,0.460844,0.404347,0.722917,0.406382,0.78845,0.407064,0.839168
73,0.0,0.73,0.349747,0.460844,0.404347,0.722917,0.406382,0.78845,0.407064,0.839168
74,0.0,0.74,0.349747,0.460844,0.404347,0.722917,0.406382,0.78845,0.407064,0.839168


In [7]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_03.csv")
df.sort_values('recall@250',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
418,0.4,0.14,0.438272,0.523776,0.502519,0.786622,0.504212,0.841174,0.504663,0.862234
426,0.4,0.22,0.432978,0.528609,0.497271,0.786122,0.499086,0.841174,0.499535,0.862234
327,0.3,0.24,0.426431,0.517066,0.490480,0.776603,0.492524,0.841174,0.492981,0.862234
413,0.4,0.09,0.438507,0.523776,0.502130,0.785084,0.504016,0.841174,0.504468,0.862234
414,0.4,0.10,0.438438,0.523776,0.502220,0.785853,0.504012,0.841174,0.504464,0.862234
...,...,...,...,...,...,...,...,...,...,...
71,0.0,0.71,0.349747,0.460844,0.404347,0.722917,0.406382,0.788450,0.407064,0.839168
72,0.0,0.72,0.349747,0.460844,0.404347,0.722917,0.406382,0.788450,0.407064,0.839168
73,0.0,0.73,0.349747,0.460844,0.404347,0.722917,0.406382,0.788450,0.407064,0.839168
74,0.0,0.74,0.349747,0.460844,0.404347,0.722917,0.406382,0.788450,0.407064,0.839168


In [12]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_01.csv")
df.sort_values('recall@250',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
444,0.4,0.40,0.298443,0.478309,0.368994,0.787436,0.371701,0.856283,0.372400,0.890197
445,0.4,0.41,0.298397,0.475809,0.369160,0.787436,0.371863,0.856283,0.372563,0.890197
446,0.4,0.42,0.298826,0.475809,0.369835,0.787436,0.372533,0.856283,0.373235,0.890197
447,0.4,0.43,0.298564,0.474559,0.369735,0.788102,0.372409,0.856283,0.373112,0.890197
448,0.4,0.44,0.298428,0.474559,0.369599,0.788102,0.372271,0.856283,0.372974,0.890197
...,...,...,...,...,...,...,...,...,...,...
70,0.0,0.70,0.226392,0.370455,0.289919,0.713360,0.293313,0.806913,0.294015,0.849995
71,0.0,0.71,0.226392,0.370455,0.289919,0.713360,0.293313,0.806913,0.294015,0.849995
72,0.0,0.72,0.226392,0.370455,0.289919,0.713360,0.293313,0.806913,0.294015,0.849995
73,0.0,0.73,0.226392,0.370455,0.289919,0.713360,0.293313,0.806913,0.294015,0.849995


# Round 4

In [4]:
queries_8b4 = subset_byId(queries, test_8b[3])

queries_8b4, queries_8b4_gs = separate_queries_goldstandard(queries_8b4, additional_keys=["limit_date"])
print(len(queries_8b4))

100


In [ ]:
import itertools
import random

## SEARCH PARAMETERS

TOP_N = [10,100, 250, 500]
K1 = list(map(lambda x:x/10, range(0,11,1)))
B = list(map(lambda x:x/100, range(0,101,1)))

RANDOM_SEARCH = False

combinations = list(itertools.product(K1,B))

if RANDOM_SEARCH:
    N_Experiments = 500
    selected = random.sample(combinations, N_Experiments)
    print(selected)
else:
    selected = combinations

count = 0
with open("search_bm25_bioasq_9b_batch_04"+".csv", "w") as fOut:
    header = "k1,b,"+",".join(sum([ ["map@"+str(x),"recall@"+str(x)] for x in TOP_N],[]))+"\n"
    fOut.write(header)
    
    for k1,beta in selected:
        
        print(count,"/",len(selected))
        
        results = execute_search(es, queries_8b4, TOP_N[-1], index_name, k1=k1, b=beta)
        results = create_document_run(queries_8b4, results)
        
        metrics_list = map(lambda x:str(x), sum([ list(evaluation(results, queries_8b4_gs, top)) for top in TOP_N],[]))
        line = f"{k1},{beta}," + ",".join(metrics_list)+"\n"
        fOut.write(line)
        fOut.flush()
        count+=1
        
        

0 / 1111
Setting the k1 and b for BM25
The inquery limit_date will be used
1 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
2 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
3 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
4 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
5 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
6 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
7 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
8 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
9 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
10 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
11 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
1

98 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
99 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
100 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
101 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
102 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
103 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
104 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
105 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
106 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
107 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
108 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
109 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will 

In [6]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_04.csv")
df.sort_values('recall@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
675,0.6,0.69,0.440270,0.579726,0.492244,0.824168,0.495077,0.864291,0.495845,0.908254
676,0.6,0.70,0.440819,0.579726,0.492706,0.824168,0.495539,0.864291,0.496307,0.908254
646,0.6,0.40,0.452761,0.578794,0.506256,0.822346,0.508643,0.866810,0.509396,0.907966
647,0.6,0.41,0.454494,0.580461,0.507371,0.822346,0.509762,0.866810,0.510514,0.907966
648,0.6,0.42,0.454427,0.580116,0.507983,0.822346,0.510379,0.866810,0.511152,0.908735
...,...,...,...,...,...,...,...,...,...,...
69,0.0,0.69,0.326439,0.500989,0.373614,0.747475,0.376194,0.802031,0.376971,0.863716
70,0.0,0.70,0.326439,0.500989,0.373614,0.747475,0.376194,0.802031,0.376971,0.863716
71,0.0,0.71,0.326439,0.500989,0.373614,0.747475,0.376194,0.802031,0.376971,0.863716
72,0.0,0.72,0.326439,0.500989,0.373614,0.747475,0.376194,0.802031,0.376971,0.863716


In [7]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_04.csv")
df.sort_values('map@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
520,0.5,0.15,0.474314,0.603401,0.496820,0.770320,0.497596,0.813694,0.498220,0.885482
519,0.5,0.14,0.474308,0.603401,0.496812,0.770320,0.497585,0.813694,0.498209,0.885482
612,0.6,0.06,0.474987,0.607226,0.496403,0.761577,0.497424,0.815574,0.498026,0.891315
618,0.6,0.12,0.474110,0.607226,0.495742,0.761577,0.496763,0.817027,0.497390,0.888815
619,0.6,0.13,0.474067,0.607226,0.495737,0.764077,0.496717,0.817027,0.497346,0.888815
...,...,...,...,...,...,...,...,...,...,...
71,0.0,0.71,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
72,0.0,0.72,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
73,0.0,0.73,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
74,0.0,0.74,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830


In [9]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_04.csv")
df.sort_values('recall@250',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
624,0.6,0.18,0.448762,0.571974,0.504066,0.816727,0.506549,0.878307,0.507262,0.907966
507,0.5,0.02,0.441877,0.588452,0.496815,0.813652,0.499253,0.877458,0.499886,0.902540
506,0.5,0.01,0.440760,0.586254,0.496099,0.813652,0.498458,0.877114,0.499132,0.900040
529,0.5,0.24,0.440398,0.570946,0.497483,0.816049,0.500148,0.877112,0.500758,0.906148
528,0.5,0.23,0.440531,0.570946,0.497457,0.816049,0.500113,0.877112,0.500719,0.906148
...,...,...,...,...,...,...,...,...,...,...
69,0.0,0.69,0.326439,0.500989,0.373614,0.747475,0.376194,0.802031,0.376971,0.863716
70,0.0,0.70,0.326439,0.500989,0.373614,0.747475,0.376194,0.802031,0.376971,0.863716
71,0.0,0.71,0.326439,0.500989,0.373614,0.747475,0.376194,0.802031,0.376971,0.863716
72,0.0,0.72,0.326439,0.500989,0.373614,0.747475,0.376194,0.802031,0.376971,0.863716


# Round 5

In [4]:
queries_8b5 = subset_byId(queries, test_8b[4])

queries_8b5, queries_8b5_gs = separate_queries_goldstandard(queries_8b5, additional_keys=["limit_date"])
print(len(queries_8b5))

100


In [ ]:
import itertools
import random

## SEARCH PARAMETERS

TOP_N = [10,100, 250, 500]
K1 = list(map(lambda x:x/10, range(0,11,1)))
B = list(map(lambda x:x/100, range(0,101,1)))

RANDOM_SEARCH = False

combinations = list(itertools.product(K1,B))

if RANDOM_SEARCH:
    N_Experiments = 500
    selected = random.sample(combinations, N_Experiments)
    print(selected)
else:
    selected = combinations

count = 0
with open("search_bm25_bioasq_9b_batch_05"+".csv", "w") as fOut:
    header = "k1,b,"+",".join(sum([ ["map@"+str(x),"recall@"+str(x)] for x in TOP_N],[]))+"\n"
    fOut.write(header)
    
    for k1,beta in selected:
        
        print(count,"/",len(selected))
        
        results = execute_search(es, queries_8b5, TOP_N[-1], index_name, k1=k1, b=beta)
        results = create_document_run(queries_8b5, results)
        
        metrics_list = map(lambda x:str(x), sum([ list(evaluation(results, queries_8b5_gs, top)) for top in TOP_N],[]))
        line = f"{k1},{beta}," + ",".join(metrics_list)+"\n"
        fOut.write(line)
        fOut.flush()
        count+=1
        

0 / 1111
Setting the k1 and b for BM25
The inquery limit_date will be used
1 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
2 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
3 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
4 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
5 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
6 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
7 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
8 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
9 / 1111query: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
10 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
11 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
1

98 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
99 / 1111uery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
100 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
101 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
102 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
103 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
104 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
105 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
106 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
107 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
108 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
109 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will 

The inquery limit_date will be used
196 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
197 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
198 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
199 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
200 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
201 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
202 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
203 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
204 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
205 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
206 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
207 / 1111ery: 80
Setting the k1 and b f

The inquery limit_date will be used
391 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
392 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
393 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
394 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
395 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
396 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
397 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
398 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
399 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
400 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
401 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
402 / 1111ery: 80
Setting the k1 and b f

The inquery limit_date will be used
586 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
587 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
588 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
589 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
590 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
591 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
592 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
593 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
594 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
595 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
596 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
597 / 1111ery: 80
Setting the k1 and b f

The inquery limit_date will be used
781 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
782 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
783 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
784 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
785 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
786 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
787 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
788 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
789 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
790 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
791 / 1111ery: 80
Setting the k1 and b for BM25
The inquery limit_date will be used
792 / 1111ery: 80
Setting the k1 and b f

In [9]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_05.csv")
df.sort_values('recall@250',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
657,0.6,0.51,0.467198,0.609485,0.488578,0.773264,0.489535,0.840194,0.489916,0.885082
658,0.6,0.52,0.466296,0.609485,0.487595,0.770764,0.488627,0.840194,0.489009,0.885082
659,0.6,0.53,0.465162,0.609485,0.486413,0.770764,0.487444,0.840194,0.487825,0.885082
660,0.6,0.54,0.465795,0.609485,0.487083,0.770764,0.488120,0.840194,0.488499,0.885082
661,0.6,0.55,0.466301,0.609485,0.487523,0.770764,0.488563,0.840194,0.488942,0.885082
...,...,...,...,...,...,...,...,...,...,...
70,0.0,0.70,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
71,0.0,0.71,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
72,0.0,0.72,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
73,0.0,0.73,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830


In [10]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_05.csv")
df.sort_values('map@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
520,0.5,0.15,0.474314,0.603401,0.496820,0.770320,0.497596,0.813694,0.498220,0.885482
519,0.5,0.14,0.474308,0.603401,0.496812,0.770320,0.497585,0.813694,0.498209,0.885482
612,0.6,0.06,0.474987,0.607226,0.496403,0.761577,0.497424,0.815574,0.498026,0.891315
618,0.6,0.12,0.474110,0.607226,0.495742,0.761577,0.496763,0.817027,0.497390,0.888815
619,0.6,0.13,0.474067,0.607226,0.495737,0.764077,0.496717,0.817027,0.497346,0.888815
...,...,...,...,...,...,...,...,...,...,...
71,0.0,0.71,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
72,0.0,0.72,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
73,0.0,0.73,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
74,0.0,0.74,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830


In [11]:
import pandas as pd
df = pd.read_csv("search_bm25_bioasq_9b_batch_05.csv")
df.sort_values('recall@100',ascending=False)

,k1,b,map@10,recall@10,map@100,recall@100,map@250,recall@250,map@500,recall@500
448,0.4,0.44,0.462964,0.600283,0.486265,0.777014,0.487021,0.819860,0.487524,0.885332
446,0.4,0.42,0.462624,0.599033,0.486386,0.777014,0.487135,0.819860,0.487638,0.885332
447,0.4,0.43,0.462464,0.599033,0.486211,0.777014,0.486963,0.819860,0.487465,0.885332
449,0.4,0.45,0.462415,0.599374,0.486261,0.777014,0.487049,0.822360,0.487522,0.885332
450,0.4,0.46,0.462415,0.599374,0.486329,0.776264,0.487095,0.822360,0.487568,0.885332
...,...,...,...,...,...,...,...,...,...,...
70,0.0,0.70,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
71,0.0,0.71,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
72,0.0,0.72,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
73,0.0,0.73,0.373241,0.526983,0.398845,0.727590,0.400037,0.791827,0.400279,0.817830
